5.0 - Chuẩn bị dữ liệu

In [2]:
import pandas as pd
df_full = pd.read_csv('../data_clean/full_merged_data.csv')
df_thong_tin = pd.read_csv('../data_clean/sensor_info_clean.csv')
df_bao_tri = pd.read_csv('../data_clean/maintenance_log_clean.csv')

5.1 - Pivot giá trị đo trung bình theo sensor_type × location

In [3]:
pivot_gia_tri = df_full.pivot_table(values='value', index='sensor_type', columns='location', aggfunc='mean')
pivot_gia_tri

location,Line1
sensor_type,
Current,38.571429


5.2 - Pivot tổng downtime_hours theo machine_id × issue_type

In [4]:
pivot_downtime = df_full.pivot_table(values='downtime_hours', index='machine_id', columns='issue_type', aggfunc='sum')
pivot_downtime

issue_type,Electrical,Overheat,Vibration
machine_id,,,
MC01,80.0,48.0,80.0


5.3 - Thực hiện stack/unstack trên bảng pivot để luyện Multindex

In [5]:
stacked = pivot_downtime.stack()
unstacked = stacked.unstack()
stacked, unstacked

(machine_id  issue_type
 MC01        Electrical    80.0
             Overheat      48.0
             Vibration     80.0
 dtype: float64,
 issue_type  Electrical  Overheat  Vibration
 machine_id                                 
 MC01              80.0      48.0       80.0)

5.4 - Xác định máy/vị trí có rủi ro cao nhất

In [6]:
so_su_co = df_bao_tri.groupby('machine_id')['issue_type'].count()
tong_downtime = df_bao_tri.groupby('machine_id')['downtime_hours'].sum()
nguy_co = pd.DataFrame({'so_su_co': so_su_co, 'tong_downtime': tong_downtime}).fillna(0) 

In [7]:
machine_location = df_thong_tin[['machine_id', 'location']].drop_duplicates().set_index('machine_id')

nguy_co = nguy_co.merge(machine_location, left_index=True, right_index=True, how='left')

may_rui_ro_cao = nguy_co.sort_values(['so_su_co', 'tong_downtime'], ascending=False)

print("Các máy và vị trí có rủi ro sự cố cao nhất (theo số sự cố và downtime):")
may_rui_ro_cao

Các máy và vị trí có rủi ro sự cố cao nhất (theo số sự cố và downtime):


,so_su_co,tong_downtime,location
machine_id,,,
MC01,17,26.0,Line1
MC01,17,26.0,Line 1
MC05,12,21.5,Line1
MC05,12,21.5,Khu A
MC05,12,21.5,Line 1
MC02,11,22.0,Khu A
MC03,10,30.0,Line1
MC04,10,24.5,Line1
MC04,10,24.5,Khu A
